In [1]:
import tensorflow as tf
import numpy as np
from network.model_fn import triplet_model_fn

### convenient class for easy usages

In [2]:
class TripletNet(object):
    def __init__(self, model_path):
        self.anchors = tf.placeholder(tf.float32, (None, 208, 208, 1), "a")
        self.positives = tf.placeholder(tf.float32, (None, 208, 208, 1), "p")
        self.negatives = tf.placeholder(tf.float32, (None, 208, 208, 1), "n")
        self.spec = triplet_model_fn(
            self.anchors, self.positives, self.negatives,
            n_feats=128, name='triplet-net', mode='TEST'
        )
        config = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False,
            intra_op_parallelism_threads=8,
            inter_op_parallelism_threads=0
        )
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config)
        self.saver = tf.train.Saver()
        
        # initialize
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(model_path)
        if ckpt and ckpt.model_checkpoint_path:
            self.saver.restore(self.sess, ckpt.model_checkpoint_path)
            
    def get_feature(self, images, batch_size=16):
        image_features = []
        N = len(images)
        batch_iters = (N + batch_size) // batch_size
        for i in range(batch_iters):
            feed_dict = dict(self.spec.test_feed_dict)
            feed_dict[self.anchors] = images[i*batch_size:(i+1)*batch_size]
            features = self.sess.run(self.spec.a_feat, feed_dict=feed_dict)
            image_features.append(features)
        image_features = np.concatenate(image_features, axis=0)
        return image_features
    
    def get_all_outputs(self, images):
        layers = ['conv-{:d}'.format(i+1) for i in range(11)]
        layers.append('merged')
        layers.append('conv-12')
        layers.append('conv-13')
        layers.append('fc1')
        
        ops = [
            self.spec.net.conv1, self.spec.net.conv2, self.spec.net.conv3,
            self.spec.net.conv4, self.spec.net.conv5, self.spec.net.conv6,
            self.spec.net.conv7, self.spec.net.conv8, self.spec.net.conv9,
            self.spec.net.conv10, self.spec.net.conv11, 
            self.spec.net.merged,
            self.spec.net.conv12, self.spec.net.conv13,  
            self.spec.net.fc1
        ]
        
        outputs = []
        for i in range(len(images)):
            feed_dict = dict(self.spec.test_feed_dict)
            
            im = images[i]
            im = np.expand_dims(im, axis=0)
            
            feed_dict[self.anchors] = im
            output = self.sess.run(ops, feed_dict=feed_dict)
            
            output_dict = dict()
            for layer, o in zip(layers, output):
                output_dict[layer] = o
            outputs.append(output_dict)
        return outputs
        

In [3]:
model_path = './log/ms_softmargin_f128_lr'

In [4]:
# load network
net = TripletNet(model_path)

INFO:tensorflow:Restoring parameters from ./log/ms_softmargin_f128_lr/triplet_net-255


In [5]:
# get features
images = np.random.random((1, 208, 208, 1)).astype(np.float32)
features = net.get_feature(images)
print("feature dim: ", features.shape)

feature dim:  (1, 128)


In [6]:
# get tensors by an random example
outputs = net.get_all_outputs(images)

outputs = outputs[0]
for k, v in outputs.items():
    print("{}: {}".format(k, v.shape))

conv-1: (1, 208, 208, 32)
conv-2: (1, 104, 104, 64)
conv-3: (1, 104, 104, 128)
conv-4: (1, 104, 104, 64)
conv-5: (1, 52, 52, 128)
conv-6: (1, 52, 52, 256)
conv-7: (1, 52, 52, 128)
conv-8: (1, 26, 26, 256)
conv-9: (1, 26, 26, 512)
conv-10: (1, 26, 26, 256)
conv-11: (1, 13, 13, 512)
merged: (1, 13, 13, 3584)
conv-12: (1, 13, 13, 1024)
conv-13: (1, 13, 13, 512)
fc1: (1, 128)
